In [60]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/Users/Killavus/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ceil']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [61]:
import numpy as np
from scipy import ndimage
from math import ceil

In [62]:
def createColorPattern(color, colorMask):
    maskBase = map(lambda x: x == color, list(colorMask))
    return np.array(maskBase).reshape((2, 2))

def maskFromPattern(colorPattern, imageShape):
    timesW = ceil(float(imageShape[0]) / 2)
    timesH = ceil(float(imageShape[1]) / 2)

    result = np.tile(colorPattern, (timesW, timesH))

    if result.shape[0] == (imageShape[0] + 1):
        result = result[0:-1, :]

    if result.shape[1] == (imageShape[1] + 1):
        result = result[:, 0:-1]

    return result * 1

def colorLayer(rawImage, color, colorSetup):
    colorPattern = createColorPattern(color, colorSetup)
    colorMask = maskFromPattern(colorPattern, rawImage.shape)
    return rawImage * colorMask

def convoluteColorLayer(rawImage, color, colorSetup, convolutionKernel):
    return ndimage.filters.convolve(colorLayer(rawImage, color, colorSetup), convolutionKernel)
    
def makeImage(rLayer, gLayer, bLayer):
    image = np.empty(shape=(rLayer.shape[0], rLayer.shape[1], 3))
    
    image[:, :, 0] = rLayer
    image[:, :, 1] = gLayer
    image[:, :, 2] = bLayer
    
    return image

def linearKernel(color):
    if color == 'g':
        return np.array([
            [0.00,  0.25, 0.00],
            [0.25, 1.00, 0.25],
            [0.00,  0.25, 0.00]
        ], dtype=np.float)
    else:
        return np.array([
            [0.25, 0.50, 0.25],
            [0.50, 1.00, 0.50],
            [0.25, 0.50, 0.25]
        ], dtype=np.float)
    
# Input format:
# [ 0 1 2 ]
# [ 3 4 5 ]
# [ 6 7 8 ]
def redBlueEdgeDirectedInterpolation(window):
    if window[4] == 0:
        deltaTLBR = abs(window[0] - window[8])
        deltaTRBL = abs(window[2] - window[6])
        
        if deltaTLBR > deltaTRBL:
            return float(window[2] + window[6]) / 2
        elif deltaTLBR <= deltaTRBL:
            return float(window[0] + window[8]) / 2
        else:
            return float(window[0] + window[2] + window[6] + window[8]) / 4
    else:
        return window
    
def greenEdgeDirectedInterpolation(window):
    if window[4] == 0:
        deltaLR = abs(window[3] - window[5])
        deltaTD = abs(window[1] - window[7])

        if deltaLR > deltaTD:
            return float(window[1] + window[7]) / 2
        elif deltaLR < deltaTD:
            return float(window[3] + window[5]) / 2
        else:
            return float(window[1] + window[3] + window[5] + window[7]) / 4
    else:
        return window[4]

def demosaicEdgeDirected(rawImage, colorSetup):    
    layers = {}
    layers['r'] = convoluteColorLayer(rawImage, 'r', colorSetup, linearKernel('r'))
    layers['g'] = ndimage.filters.generic_filter(
                            colorLayer(rawImage, 'g', colorSetup),
                            greenEdgeDirectedInterpolation,
                            size=3)    
    layers['b'] = convoluteColorLayer(rawImage, 'b', colorSetup, linearKernel('b'))    
        
    return makeImage(layers['r'], layers['g'], layers['b'])

In [63]:
files = {
    './lighthouse_RAW_noisy_sigma0.01.png': 'rggb',
#    './raw/signs.png': 'grbg',
    './raw/signs-small.png': 'bggr',
#    './raw/text.png': 'gbrg',
#    './raw/text2.png': 'gbgr'
}


for fileName, colorSetup in files.iteritems():
    rawImage = ndimage.imread(fileName, mode="F")
    print fileName, rawImage.shape
    coloredImage = demosaicEdgeDirected(rawImage, colorSetup)
    imsave(fileName + '.edgedirected.mine.png', coloredImage / 255.0)

./lighthouse_RAW_noisy_sigma0.01.png (768, 512)
./raw/signs-small.png (520, 426)
